In [50]:
import itertools

from extract import extract_dataset_archive
from split import convert_train_set_to_images
import os
from tensorflow.keras.layers import Dense, LSTM, Activation
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import minmax_scale

# os.system('python scripts/demo_inference.py --cfg pretrained_models/256x192_res152_lr1e-3_1x-duc.yaml --checkpoint  pretrained_models/fast_421_res152_256x192.pth --video ../dataset/NonViolence/NV_1.mp4 --outdir ../dataset --sp')

def prepare_dataset():
    extract_dataset_archive()
    convert_train_set_to_images()


def convert_videos_to_json(path_in, path_out):
    os.system('python scripts/demo_inference.py --cfg pretrained_models/256x192_res152_lr1e-3_1x-duc.yaml '
              '--checkpoint  pretrained_models/fast_421_res152_256x192.pth --video ' + path_in + ' --outdir ' + path_out + ' --sp')


#prepare_dataset()
#os.chdir("AlphaPose")

# for i in range(1, 1001):
#    file_in = str("../dataset/Violence/V_") + str(i) + ".mp4"
#    file_out = str("../dataset/results/V_") + str(i) + "_json"
#    convert_videos_to_json(file_in, file_out)

#for i in range(1, 1001):
#    file_in = str("../dataset/NonViolence/NV_") + str(i) + ".mp4"
#    file_out = str("../dataset/results/NV_") + str(i) + "_json"
#    convert_videos_to_json(file_in, file_out)



person_count = 5


In [38]:
model = Sequential()
model.add(LSTM(100, input_shape=(150, 2048))) # 150, 2048 = n_chunks, chunk_size
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dense(50))
model.add(Activation('sigmoid'))
model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

csv_dir = "dataset/csvs"

csv_files = os.listdir(csv_dir)

train_files, test_files = train_test_split(csv_files, train_size=0.8)

In [57]:
for file_name in train_files:
    csv = pd.read_csv(csv_dir + "/" + file_name)
    
    y_train = np.array(csv.violent)
    
    csv = csv.drop('violent',axis=1)
    
    labels = csv.columns.values
    print(labels[2:])
    
    
    for label in labels:
        minmax_scale = preprocessing.MinMaxScaler().fit(csv[[label]])
        csv[[label]] = minmax_scale.transform(csv[[label]])
    
#     column_trans = ColumnTransformer(
#         [('scaler', MinMaxScaler(),list(range(2,70)))],
#         remainder='passthrough') 
#     column_trans.fit_transform(csv)
    
    data = csv.to_numpy()


    val_cumulative = {}
    frame_ids = set()
    for person in data:
        frame_ids.add(person[0])
        frame, person_id = person[:2]

        vel_array = []
        for vel in range(5, 70, 4):
            vel_array.append(vel)

        total_vel = sum(vel_array)

        if person_id in val_cumulative:
            val_cumulative[person_id] += total_vel
        else:
            val_cumulative[person_id] = 0

    val_cumulative = {k: v for k, v in sorted(val_cumulative.items(), key=lambda item: item[1], reverse=True)}
    top_people = dict(itertools.islice(val_cumulative.items(), person_count))
    top_people_dataset = []

    for frame_id in frame_ids:
        top_people_dataset.append([])
        people = [item for item in data if item[0] == frame_id]
        for person in people:
            if person[1] in top_people:
                top_people_dataset[int(person[0])].append(person[2:])
            else:
                top_people_dataset[int(person[0])].append(np.zeros(68))

#     print(y_train.shape)
    
    output = top_people_dataset
    top_people_dataset = np.asarray(top_people_dataset)
#     print(top_people_dataset)
#     top_people_dataset = np.delete(top_people_dataset, [0, 1, -1], axis=1)
    
    

output
#     for i in top_people_dataset.size:
#         top_people_dataset = scaler.transform(top_people_dataset[])
    
#     output = preprocessing.scale()

#     x_train = np.array(csv.drop(['frame', 'object_id', 'violent'],axis=1))
#     print(x_train.shape)

#     model.train_on_batch(x_train, y_train)

['NoseX' 'NoseY' 'NoseC' 'NoseV' 'LEyeX' 'LEyeY' 'LEyeC' 'LEyeV' 'REyeX'
 'REyeY' 'REyeC' 'REyeV' 'LEarX' 'LEarY' 'LEarC' 'LEarV' 'REarX' 'REarY'
 'REarC' 'REarV' 'LShoulderX' 'LShoulderY' 'LShoulderC' 'LShoulderV'
 'RShoulderX' 'RShoulderY' 'RShoulderC' 'RShoulderV' 'LElbowX' 'LElbowY'
 'LElbowC' 'LElbowV' 'RElbowX' 'RElbowY' 'RElbowC' 'RElbowV' 'LWristX'
 'LWristY' 'LWristC' 'LWristV' 'RWristX' 'RWristY' 'RWristC' 'RWristV'
 'LHipX' 'LHipY' 'LHipC' 'LHipV' 'RHipX' 'RHipY' 'RHipC' 'RHipV' 'LKneeX'
 'LKneeY' 'LKneeC' 'LKneeV' 'RkneeX' 'RkneeY' 'RkneeC' 'RkneeV' 'LAnkleX'
 'LAnkleY' 'LAnkleC' 'LAnkleV' 'RAnkleX' 'RAnkleY' 'RAnkleC' 'RAnkleV']
['NoseX' 'NoseY' 'NoseC' 'NoseV' 'LEyeX' 'LEyeY' 'LEyeC' 'LEyeV' 'REyeX'
 'REyeY' 'REyeC' 'REyeV' 'LEarX' 'LEarY' 'LEarC' 'LEarV' 'REarX' 'REarY'
 'REarC' 'REarV' 'LShoulderX' 'LShoulderY' 'LShoulderC' 'LShoulderV'
 'RShoulderX' 'RShoulderY' 'RShoulderC' 'RShoulderV' 'LElbowX' 'LElbowY'
 'LElbowC' 'LElbowV' 'RElbowX' 'RElbowY' 'RElbowC' 'RElbowV' '

[[array([0.80905378, 0.13505539, 0.98724041, 0.        , 0.87813757,
         0.11346228, 0.96972299, 0.        , 0.76182747, 0.11400993,
         0.98056129, 0.        , 0.87798993, 0.12624997, 0.89693937,
         0.        , 0.65707842, 0.16435107, 0.91985695, 0.        ,
         0.76005635, 0.42878996, 0.86014148, 0.        , 0.51968586,
         0.47843017, 0.88156156, 0.        , 0.74514429, 0.73209105,
         0.57668258, 0.        , 0.44610314, 0.8622484 , 0.9706476 ,
         0.        , 0.81328352, 0.96310208, 0.60767221, 0.        ,
         0.30403028, 0.54966052, 0.90427029, 0.        , 0.72883209,
         0.6952719 , 0.58483051, 0.        , 0.62740673, 0.65158747,
         0.64920863, 0.        , 0.62015363, 0.91549578, 0.50385784,
         0.        , 0.37713384, 0.89754405, 0.43431295, 0.        ,
         0.6794483 , 0.76486175, 0.14133701, 0.        , 0.40717989,
         0.53491414, 0.06419051, 0.        ]),
  array([0.86825854, 0.07569542, 0.59554772, 0.9692539 ,

In [17]:
# x_train = np.array(x_train)
# print(x_train.shape())




# x_train = []
# y_train = []
# epochs = 1500
# batchs = 100
# for i in top_people_dataset:
#     x_train.append(i[0])
#     y_train.append(np.array(i[1]))

# model.fit(x_train, y_train, epochs, batch_size=batchs, verbose=1)
# model.save("rnn.h5", overwrite=True)

In [58]:
output

[[array([0.80905378, 0.13505539, 0.98724041, 0.        , 0.87813757,
         0.11346228, 0.96972299, 0.        , 0.76182747, 0.11400993,
         0.98056129, 0.        , 0.87798993, 0.12624997, 0.89693937,
         0.        , 0.65707842, 0.16435107, 0.91985695, 0.        ,
         0.76005635, 0.42878996, 0.86014148, 0.        , 0.51968586,
         0.47843017, 0.88156156, 0.        , 0.74514429, 0.73209105,
         0.57668258, 0.        , 0.44610314, 0.8622484 , 0.9706476 ,
         0.        , 0.81328352, 0.96310208, 0.60767221, 0.        ,
         0.30403028, 0.54966052, 0.90427029, 0.        , 0.72883209,
         0.6952719 , 0.58483051, 0.        , 0.62740673, 0.65158747,
         0.64920863, 0.        , 0.62015363, 0.91549578, 0.50385784,
         0.        , 0.37713384, 0.89754405, 0.43431295, 0.        ,
         0.6794483 , 0.76486175, 0.14133701, 0.        , 0.40717989,
         0.53491414, 0.06419051, 0.        ]),
  array([0.86825854, 0.07569542, 0.59554772, 0.9692539 ,

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])